In [1]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np

In [3]:
folder = 'D:/Dataset/HighResolutionPopulation/'
country = 'KSA'
pop_file = folder + country + '/pop.csv'
bs_file = folder + country + '/BS.csv'
poly_file = folder + country + '/poly/poly.shp'
grid_file = folder + country + '/grid.geojson'

In [4]:
print('Loading population, bs, poly, and grid data...')
df_pop = pd.read_csv(pop_file)
df_bs = pd.read_csv(bs_file)
gdf_poly = gpd.read_file(poly_file)
gdf_poly.set_crs('epsg:4326', inplace=True)
grid = json.load(open(grid_file))
print('Creating GeoDataFrame from grid data...')
df_grid = gpd.GeoDataFrame.from_features(grid['features'])
df_grid['id'] = range(df_grid.shape[0])
df_grid.set_crs('epsg:4326', inplace=True)

Loading population, bs, poly, and grid data...
Creating GeoDataFrame from grid data...


,geometry,id
0,"POLYGON ((34.49430 32.27078, 34.56488 32.27078...",0
1,"POLYGON ((34.49430 32.21686, 34.56488 32.21686...",1
2,"POLYGON ((34.49430 32.16294, 34.56488 32.16294...",2
3,"POLYGON ((34.49430 32.10902, 34.56488 32.10902...",3
4,"POLYGON ((34.49430 32.05510, 34.56488 32.05510...",4
...,...,...
89995,"POLYGON ((55.59601 16.36391, 55.66659 16.36391...",89995
89996,"POLYGON ((55.59601 16.30999, 55.66659 16.30999...",89996
89997,"POLYGON ((55.59601 16.25607, 55.66659 16.25607...",89997
89998,"POLYGON ((55.59601 16.20215, 55.66659 16.20215...",89998


In [5]:
print('Creating GeoDataFrame from population data...')
gdf_population = gpd.GeoDataFrame(df_pop,
                                  geometry=gpd.points_from_xy(df_pop.Lon, df_pop.Lat))

Creating GeoDataFrame from population data...


In [6]:
gdf_population.set_crs('epsg:4326', inplace=True)
print('Creating GeoDataFrame from bs data...')
gdf_bs = gpd.GeoDataFrame(df_bs,
                          geometry=gpd.points_from_xy(df_bs.lon, df_bs.lat))
gdf_bs.set_crs('epsg:4326', inplace=True)

Creating GeoDataFrame from bs data...


,radio,mcc,net,area,cell,unit,lon,lat,range,samples,changeable,created,updated,averageSignal,geometry
0,GSM,420,1,338,56308,0,41.951981,19.531631,1000,9,1,1459810864,1466902057,0,POINT (41.95198 19.53163)
1,GSM,420,1,457,32439,0,46.680644,24.698692,1000,13,1,1459786728,1492150097,0,POINT (46.68064 24.69869)
2,GSM,420,3,2149,27691,0,41.405411,18.736496,1000,1,1,1459676483,1459676483,0,POINT (41.40541 18.73650)
3,GSM,420,4,22101,12793,0,46.687637,24.701660,1000,10,1,1459807892,1474807714,0,POINT (46.68764 24.70166)
4,GSM,420,1,74,21891,0,39.172440,21.564789,1000,2,1,1459688718,1477201059,0,POINT (39.17244 21.56479)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335984,LTE,420,3,42461,57826838,0,39.269676,21.466070,1000,1,1,1582469837,1582469837,0,POINT (39.26968 21.46607)
335985,LTE,420,3,12461,57826858,0,39.269672,21.465914,1000,1,1,1582475615,1582475615,0,POINT (39.26967 21.46591)
335986,LTE,420,3,1930,56300293,0,39.270645,21.464865,1000,1,1,1584464403,1584464403,0,POINT (39.27065 21.46486)
335987,LTE,420,3,42461,57294132,0,39.270660,21.464842,1000,1,1,1587491626,1587491626,0,POINT (39.27066 21.46484)


In [7]:
print('Calculating # of populations per grid...')
pop_in_grid = gpd.sjoin(gdf_population, df_grid, op='within')
print('Calculating # of bs per grid...')
bs_in_grid = gpd.sjoin(gdf_bs, df_grid, op='within')

Calculating # of populations per grid...
Calculating # of bs per grid...


In [22]:
bs_in_grid.drop(['index_right'], axis=1, inplace=True)

In [26]:
print('Removing the BS that are out of the poly...')
bs_in_poly = gpd.sjoin(bs_in_grid, gdf_poly, op='within')


Removing the BS that are out of the poly...


In [27]:
bs_in_poly = bs_in_poly[['radio', 'net', 'cell', 'lon', 'lat', 'geometry', 'id']]

In [28]:
bs_all_count = bs_in_poly[['id', 'geometry']].groupby('id').count().reset_index()

In [29]:
bs_all_count

,id,geometry
0,970,4
1,1854,2
2,1855,116
3,1856,16
4,1857,1
...,...,...
6029,82981,1
6030,83279,2
6031,83280,3
6032,83579,5


In [278]:
df_grid_tunisia['id'] = range(df_grid_tunisia.shape[0])

In [279]:
df_grid_tunisia.head()

,geometry,id
0,"POLYGON ((34.49430 32.27078, 34.56488 32.27078...",0
1,"POLYGON ((34.49430 32.21686, 34.56488 32.21686...",1
2,"POLYGON ((34.49430 32.16294, 34.56488 32.16294...",2
3,"POLYGON ((34.49430 32.10902, 34.56488 32.10902...",3
4,"POLYGON ((34.49430 32.05510, 34.56488 32.05510...",4


In [280]:
df_pop_tunisia.head()

,Lat,Lon,Population
0,27.000139,49.647917,6.226124
1,27.010417,49.657361,6.226124
2,26.984583,49.657639,6.226124
3,26.993194,49.659306,6.226124
4,27.003194,49.657917,6.226124


In [281]:
df_bs_tunisia.head()

,radio,mcc,net,area,cell,unit,lon,lat,range,samples,changeable,created,updated,averageSignal
0,GSM,420,1,338,56308,0,41.951981,19.531631,1000,9,1,1459810864,1466902057,0
1,GSM,420,1,457,32439,0,46.680644,24.698692,1000,13,1,1459786728,1492150097,0
2,GSM,420,3,2149,27691,0,41.405411,18.736496,1000,1,1,1459676483,1459676483,0
3,GSM,420,4,22101,12793,0,46.687637,24.701660,1000,10,1,1459807892,1474807714,0
4,GSM,420,1,74,21891,0,39.172440,21.564789,1000,2,1,1459688718,1477201059,0


In [282]:
gdf_population = gpd.GeoDataFrame(df_pop_tunisia, geometry=gpd.points_from_xy(df_pop_tunisia.Lon, df_pop_tunisia.Lat))

In [283]:
gdf_population.head()

,Lat,Lon,Population,geometry
0,27.000139,49.647917,6.226124,POINT (49.64792 27.00014)
1,27.010417,49.657361,6.226124,POINT (49.65736 27.01042)
2,26.984583,49.657639,6.226124,POINT (49.65764 26.98458)
3,26.993194,49.659306,6.226124,POINT (49.65931 26.99319)
4,27.003194,49.657917,6.226124,POINT (49.65792 27.00319)


In [284]:
gdf_bs = gpd.GeoDataFrame(df_bs_tunisia, geometry=gpd.points_from_xy(df_bs_tunisia.lon, df_bs_tunisia.lat))

In [285]:
gdf_bs.set_crs('epsg:4326',inplace=True)
gdf_bs.head()

,radio,mcc,net,area,cell,unit,lon,lat,range,samples,changeable,created,updated,averageSignal,geometry
0,GSM,420,1,338,56308,0,41.951981,19.531631,1000,9,1,1459810864,1466902057,0,POINT (41.95198 19.53163)
1,GSM,420,1,457,32439,0,46.680644,24.698692,1000,13,1,1459786728,1492150097,0,POINT (46.68064 24.69869)
2,GSM,420,3,2149,27691,0,41.405411,18.736496,1000,1,1,1459676483,1459676483,0,POINT (41.40541 18.73650)
3,GSM,420,4,22101,12793,0,46.687637,24.701660,1000,10,1,1459807892,1474807714,0,POINT (46.68764 24.70166)
4,GSM,420,1,74,21891,0,39.172440,21.564789,1000,2,1,1459688718,1477201059,0,POINT (39.17244 21.56479)


In [286]:
gdf_population

,Lat,Lon,Population,geometry
0,27.000139,49.647917,6.226124,POINT (49.64792 27.00014)
1,27.010417,49.657361,6.226124,POINT (49.65736 27.01042)
2,26.984583,49.657639,6.226124,POINT (49.65764 26.98458)
3,26.993194,49.659306,6.226124,POINT (49.65931 26.99319)
4,27.003194,49.657917,6.226124,POINT (49.65792 27.00319)
...,...,...,...,...
5279297,24.448750,39.605972,7.864090,POINT (39.60597 24.44875)
5279298,24.445972,39.608472,7.864090,POINT (39.60847 24.44597)
5279299,24.468472,39.636806,7.864090,POINT (39.63681 24.46847)
5279300,24.453194,39.612917,7.864090,POINT (39.61292 24.45319)


In [287]:
pointInPoly = gpd.sjoin(gdf_population, df_grid_tunisia, op='within') 

In [288]:
pointInPoly.head()

,Lat,Lon,Population,geometry,index_right,id
0,27.000139,49.647917,6.226124,POINT (49.64792 27.00014),64297,64297
1,27.010417,49.657361,6.226124,POINT (49.65736 27.01042),64297,64297
3,26.993194,49.659306,6.226124,POINT (49.65931 26.99319),64297,64297
4,27.003194,49.657917,6.226124,POINT (49.65792 27.00319),64297,64297
5,27.010694,49.613194,6.226124,POINT (49.61319 27.01069),64297,64297


In [289]:
bsInPoly = gpd.sjoin(gdf_bs, df_grid_tunisia, op='within') 

C:\Users\ZHANC0C\AppData\Local\Continuum\anaconda3\envs\tbb\lib\site-packages\ipykernel_launcher.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None

  """Entry point for launching an IPython kernel.


In [290]:
bsInPoly.head()

,radio,mcc,net,area,cell,unit,lon,lat,range,samples,changeable,created,updated,averageSignal,geometry,index_right,id
0,GSM,420,1,338,56308,0,41.951981,19.531631,1000,9,1,1459810864,1466902057,0,POINT (41.95198 19.53163),31736,31736
121594,UMTS,420,1,5123,14610915,0,41.951551,19.529357,2224,32,1,1458833369,1469786707,0,POINT (41.95155 19.52936),31736,31736
199826,UMTS,420,1,532,14590071,0,41.953354,19.542618,1000,1,1,1451265919,1451265919,0,POINT (41.95335 19.54262),31736,31736
208967,UMTS,420,4,61009,49248657,0,41.951981,19.531631,1000,2,1,1459746465,1463961402,0,POINT (41.95198 19.53163),31736,31736
208972,UMTS,420,1,5363,30569379,0,41.946487,19.520645,1000,2,1,1459768377,1461555966,0,POINT (41.94649 19.52065),31736,31736


In [291]:
bs_in_poly = gpd.sjoin(gdf_bs, pp,op='within')

In [292]:
bs_in_poly = bs_in_poly[['radio', 'net', 'cell', 'lon', 'lat', 'geometry']]

In [293]:
bs_in_poly.to_file('d:/ksa_bs.geojson', driver='GeoJSON')

In [294]:
bs_poly_GSM = bsInPoly[bsInPoly['radio']=='GSM']
bs_poly_UMTS = bsInPoly[bsInPoly['radio']=='UMTS']
bs_poly_LTE = bsInPoly[bsInPoly['radio']=='LTE']

In [295]:
bs_poly_GSM = bs_poly_GSM[['id', 'geometry']]
bs_poly_UMTS = bs_poly_UMTS[['id', 'geometry']]
bs_poly_LTE = bs_poly_LTE[['id', 'geometry']]

In [296]:
bs_GSM_count = bs_poly_GSM.groupby('id').count().reset_index()

In [297]:
bs_UMTS_count = bs_poly_UMTS.groupby('id').count().reset_index()
bs_LTE_count = bs_poly_LTE.groupby('id').count().reset_index()


In [298]:
bs_all_count = bsInPoly[['id', 'geometry']].groupby('id').count().reset_index()

In [299]:
pointInPoly = pointInPoly[['id', 'geometry']]
pop_count = pointInPoly.groupby('id').count().reset_index()

In [300]:
pop_count.head()

,id,geometry
0,375,4
1,377,30
2,378,5
3,674,8
4,675,1


In [301]:
pop_align = pd.merge(left=pop_count, right=df_grid_tunisia, left_on='id', right_on='id', how='right')

In [302]:
bs_align = pd.merge(left=bs_all_count, right=df_grid_tunisia, left_on='id', right_on='id', how='right')

In [303]:
pop_align.fillna(0, inplace=True)

In [304]:
bs_align.fillna(0, inplace=True)

In [305]:
pop_align.columns = ['id', 'pop', 'geometry']

In [306]:
bs_align.columns = ['id', 'bs', 'geometry']

In [307]:
pop_align.head()

,id,pop,geometry
0,0,0.0,"POLYGON ((34.49430 32.27078, 34.56488 32.27078..."
1,1,0.0,"POLYGON ((34.49430 32.21686, 34.56488 32.21686..."
2,2,0.0,"POLYGON ((34.49430 32.16294, 34.56488 32.16294..."
3,3,0.0,"POLYGON ((34.49430 32.10902, 34.56488 32.10902..."
4,4,0.0,"POLYGON ((34.49430 32.05510, 34.56488 32.05510..."


In [308]:
bs_align.head()

,id,bs,geometry
0,0,0.0,"POLYGON ((34.49430 32.27078, 34.56488 32.27078..."
1,1,0.0,"POLYGON ((34.49430 32.21686, 34.56488 32.21686..."
2,2,0.0,"POLYGON ((34.49430 32.16294, 34.56488 32.16294..."
3,3,0.0,"POLYGON ((34.49430 32.10902, 34.56488 32.10902..."
4,4,0.0,"POLYGON ((34.49430 32.05510, 34.56488 32.05510..."


In [312]:
pop_results = gpd.GeoDataFrame(pop_align)
pop_results.set_crs('epsg:4326',inplace=True)
# pop_final = gpd.sjoin(pop_results, pp,op='within')[['id_left', 'pop', 'geometry']]
pop_final = gpd.sjoin(pop_results, pp,op='within')[['id', 'pop', 'geometry']]

In [313]:
df_bounds = pop_final['geometry'].bounds
x = (df_bounds['minx'] + df_bounds['maxx']) / 2
y = (df_bounds['miny'] + df_bounds['maxy']) / 2
pop_final['lon'] = x.values
pop_final['lat'] = y.values
pop_final.to_file('d:/ksa_pop.geojson', driver='GeoJSON')

In [314]:
bs_results = gpd.GeoDataFrame(bs_align)
bs_results.set_crs('epsg:4326',inplace=True)
# bs_final = gpd.sjoin(bs_results, pp,op='within')[['id_left', 'bs', 'geometry']]
bs_final = gpd.sjoin(bs_results, pp,op='within')[['id', 'bs', 'geometry']]

In [315]:
df_bounds = pop_final['geometry'].bounds
x = (df_bounds['minx'] + df_bounds['maxx']) / 2
y = (df_bounds['miny'] + df_bounds['maxy']) / 2
bs_final['lat'] = x.values
bs_final['lon'] = y.values
bs_final.to_file('d:/ksa_bs.geojson', driver='GeoJSON')

In [316]:
bs_final

,id,bs,geometry,lat,lon
974,974,0.0,"POLYGON ((34.70603 28.28058, 34.77660 28.28058...",34.741315,28.253624
975,975,0.0,"POLYGON ((34.70603 28.22666, 34.77660 28.22666...",34.741315,28.199703
1271,1271,0.0,"POLYGON ((34.77660 28.44235, 34.84718 28.44235...",34.811889,28.415388
1272,1272,0.0,"POLYGON ((34.77660 28.38843, 34.84718 28.38843...",34.811889,28.361467
1273,1273,0.0,"POLYGON ((34.77660 28.33451, 34.84718 28.33451...",34.811889,28.307545
...,...,...,...,...,...
89589,89589,0.0,"POLYGON ((55.52544 22.07960, 55.59601 22.07960...",55.560727,22.052641
89590,89590,0.0,"POLYGON ((55.52544 22.02568, 55.59601 22.02568...",55.560727,21.998719
89591,89591,0.0,"POLYGON ((55.52544 21.97176, 55.59601 21.97176...",55.560727,21.944798
89592,89592,0.0,"POLYGON ((55.52544 21.91784, 55.59601 21.91784...",55.560727,21.890876


In [317]:
pop_final['bs'] = bs_final['bs'].values

In [414]:
all_final = pop_final.copy()

In [415]:
# all_final['pop'] = all_final['pop'] / all_final['pop'].max()
# all_final['bs'] = norm_final['bs']  / norm_final['bs'].max()

In [416]:
user_per_bs = 100
all_final['imbalance_index'] = all_final['pop'] / (all_final['bs'] * user_per_bs)
# all_final['imbalance_index'] = all_final['pop'] / all_final['bs']

In [417]:
all_final

,id,pop,geometry,lon,lat,bs,imbalance_index
974,974,0.0,"POLYGON ((34.70603 28.28058, 34.77660 28.28058...",34.741315,28.253624,0.0,NaN
975,975,0.0,"POLYGON ((34.70603 28.22666, 34.77660 28.22666...",34.741315,28.199703,0.0,NaN
1271,1271,40.0,"POLYGON ((34.77660 28.44235, 34.84718 28.44235...",34.811889,28.415388,0.0,inf
1272,1272,0.0,"POLYGON ((34.77660 28.38843, 34.84718 28.38843...",34.811889,28.361467,0.0,NaN
1273,1273,2.0,"POLYGON ((34.77660 28.33451, 34.84718 28.33451...",34.811889,28.307545,0.0,inf
...,...,...,...,...,...,...,...
89589,89589,0.0,"POLYGON ((55.52544 22.07960, 55.59601 22.07960...",55.560727,22.052641,0.0,NaN
89590,89590,0.0,"POLYGON ((55.52544 22.02568, 55.59601 22.02568...",55.560727,21.998719,0.0,NaN
89591,89591,0.0,"POLYGON ((55.52544 21.97176, 55.59601 21.97176...",55.560727,21.944798,0.0,NaN
89592,89592,0.0,"POLYGON ((55.52544 21.91784, 55.59601 21.91784...",55.560727,21.890876,0.0,NaN


In [418]:
all_final.loc[(all_final['pop']==0) & (all_final['bs']==0), 'imbalance_index'] = 0.0

In [419]:
# all_final.loc[all_final['imbalance_index']==np.inf, 'imbalance_index']
inf_imbalance = all_final.loc[all_final['imbalance_index']==np.inf, 'pop'] / user_per_bs
# inf_imbalance = all_final.loc[all_final['imbalance_index']==np.inf, 'pop']
all_final.loc[all_final['imbalance_index']==np.inf, 'imbalance_index'] = inf_imbalance.values

In [420]:
all_final.loc[all_final['bs']>=3, 'imbalance_index'] = 0.01

In [421]:
# user_threshold = 30
# zero_imbalance = all_final.loc[all_final['imbalance_index']==np.inf, 'pop'] / user_per_bs
# all_final.loc[all_final['imbalance_index']==np.inf, 'imbalance_index'] = 0

In [422]:
# # all_final.loc[all_final['imbalance_index']==np.inf, 'imbalance_index']
# inf_imbalance = all_final.loc[all_final['imbalance_index']==np.inf, 'pop'] / user_per_bs
# all_final.loc[all_final['imbalance_index']==np.inf, 'imbalance_index'] = inf_imbalance.values

In [423]:
all_final.to_file('d:/ksa_all_info_0.01_3.geojson', driver='GeoJSON')

In [378]:
bs_final.loc[(bs_final['bs']>5) & (bs_final['bs']>0)]

,id,bs,geometry,lat,lon
2158,2158,7.0,"POLYGON ((34.98833 29.14333, 35.05890 29.14333...",35.023612,29.116369
2170,2170,7.0,"POLYGON ((34.98833 28.49627, 35.05890 28.49627...",35.023612,28.469310
2171,2171,6.0,"POLYGON ((34.98833 28.44235, 35.05890 28.44235...",35.023612,28.415388
2761,2761,9.0,"POLYGON ((35.12947 28.98156, 35.20005 28.98156...",35.164760,28.954605
3062,3062,7.0,"POLYGON ((35.20005 28.92764, 35.27062 28.92764...",35.235335,28.900683
...,...,...,...,...,...
72450,72450,42.0,"POLYGON ((51.50270 24.18254, 51.57328 24.18254...",51.537993,24.155583
79677,79677,6.0,"POLYGON ((53.19649 22.72666, 53.26706 22.72666...",53.231775,22.699700
79977,79977,8.0,"POLYGON ((53.26706 22.72666, 53.33764 22.72666...",53.302350,22.699700
82680,82680,6.0,"POLYGON ((53.90223 22.56490, 53.97281 22.56490...",53.937518,22.537935


In [375]:
bs_final[bs_final['bs']>0].shape

(5676, 5)

In [379]:
pop_final.loc[bs_final['bs']>5]

,id,pop,geometry,lon,lat,bs
2158,2158,98.0,"POLYGON ((34.98833 29.14333, 35.05890 29.14333...",35.023612,29.116369,7.0
2170,2170,1420.0,"POLYGON ((34.98833 28.49627, 35.05890 28.49627...",35.023612,28.469310,7.0
2171,2171,1210.0,"POLYGON ((34.98833 28.44235, 35.05890 28.44235...",35.023612,28.415388,6.0
2761,2761,35.0,"POLYGON ((35.12947 28.98156, 35.20005 28.98156...",35.164760,28.954605,9.0
3062,3062,40.0,"POLYGON ((35.20005 28.92764, 35.27062 28.92764...",35.235335,28.900683,7.0
...,...,...,...,...,...,...
72450,72450,0.0,"POLYGON ((51.50270 24.18254, 51.57328 24.18254...",51.537993,24.155583,42.0
79677,79677,28.0,"POLYGON ((53.19649 22.72666, 53.26706 22.72666...",53.231775,22.699700,6.0
79977,79977,55.0,"POLYGON ((53.26706 22.72666, 53.33764 22.72666...",53.302350,22.699700,8.0
82680,82680,606.0,"POLYGON ((53.90223 22.56490, 53.97281 22.56490...",53.937518,22.537935,6.0
